In [1]:
import itertools
import os
import shutil
import sys
from io import StringIO

import pandas as pd
import pytorch_lightning as pl

from coco_eval import CocoEvaluator
from detr_config import Config
from detr_dataset import InBreastDataset, collate_fn, detr_processor
from detr_detection import prepare_for_coco_detection
from detr_model import DETRModel
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from transformers import DetrConfig, DetrForObjectDetection

STDOUT = sys.stdout

The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [2]:

# HyperParameters

hyperparameters = itertools.product(*[
    Config.BACKBONES,
    Config.NUM_QUERIES,
    Config.D_MODEL,
    Config.TRANSFORMER_LAYERS,
])

hyperparameters = itertools.product(*[
    ['efficientnet_b0.ra_in1k'],
    [25],
    [64],
    [2],
])


# Hyperparameter Search

for backbone, num_queries, d_model, transformer_layers in hyperparameters:
    print('(Num Queries, Dim model, Enc-Dec Layers): ', 
            f'({num_queries}, {d_model}, {transformer_layers})' )

    # Model Construction

    config = DetrConfig.from_pretrained(
        Config.CHECKPOINT,
        num_labels=2,
        id2label = {0:'Mass', 1: 'No-Mass'}, 
        label2id = {'Mass': 0, 'No-Mass': 1},
        num_queries = num_queries,
        d_model = d_model,
        num_head = 8,
        encoder_layers = transformer_layers,
        decoder_layers = transformer_layers,
        position_embedding_type  = 'sine',
        decoder_ffn_dim = 2048,
        encoder_ffn_dim = 2048,
        backbone=backbone
    )

    model_name = [
        f'backbone={backbone.split(".")[0]}',
        f'queries={num_queries}',
        f'dmodel={d_model}',
        f'layers={transformer_layers}'
    ]
    
    logs_dir = 'lightning_logs'
    model_name = '_'.join(model_name)
    
    metrics_by_fold = []

    # K-fold Cross Validation 

    for fold in range(1,11):
        print(f"Fold {fold}")
        
        # Model
        
        detr_model = DetrForObjectDetection.from_pretrained(
            Config.CHECKPOINT,
            config = config,
            ignore_mismatched_sizes=True
        )

        model = DETRModel(detr_model=detr_model)
        
        # Datasets
        
        fold_dir = os.path.join(Config.DATASET, f'fold_{fold}')
        
        train_dataset = InBreastDataset(
            images_path = os.path.join(fold_dir, 'train'),
            processor=detr_processor
        )
        
        train_loader = DataLoader(
            dataset = train_dataset,
            batch_size = Config.BATCH_SIZE,
            collate_fn = collate_fn,
        )

        valid_dataset = InBreastDataset(
            images_path = os.path.join(fold_dir, 'valid'),
            processor=detr_processor
        )

        valid_loader = DataLoader(
            dataset = valid_dataset,
            batch_size = Config.BATCH_SIZE,
            collate_fn = collate_fn,
        )
        
        # Training

        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            save_top_k = 1,
            save_last = True,
            monitor = "valid_loss",
            mode = "min"
        )

        early_stopping_callback = pl.callbacks.EarlyStopping(
            monitor = 'valid_loss',
            patience = 15
        )

        version = os.path.join(model_name, f'fold_{fold}')

        logger = pl.loggers.TensorBoardLogger(
            save_dir = './',
            version = version
        )

        trainer = Trainer(
            # max_epochs = Config.EPOCHS, 
            max_epochs = 1,
            log_every_n_steps = 5, 
            callbacks = [
                checkpoint_callback, 
                early_stopping_callback
            ],
            accelerator = Config.ACCELERATOR,
            logger = logger
        )
        
        trainer.fit(
            model, 
            train_dataloaders = train_loader, 
            val_dataloaders = valid_loader
        )
        
        
        # Evaluation
        
        checkpoints_dir = os.path.join(logs_dir, version, 'checkpoints')
        best_checkpoint = [f for f in os.listdir(checkpoints_dir) if 'last' not in f][0]
        checkpoint_path = os.path.join(checkpoints_dir, best_checkpoint)
        
        model = DETRModel.load_from_checkpoint(checkpoint_path)

        evaluator = CocoEvaluator(
            coco_gt=valid_dataset.coco, 
            iou_types=["bbox"]
        )
        
        for batch in valid_loader:
            outputs = model(batch['pixel_values'])
            predictions = detr_processor.post_process_object_detection(outputs, threshold=0.1)
            image_ids = [label['image_id'].item() for label in batch['labels']]
            predictions = {image_id:output for image_id, output in zip(image_ids, predictions)}
            predictions = prepare_for_coco_detection(predictions)
            evaluator.update(predictions)
            
        evaluator.synchronize_between_processes()
        evaluator.accumulate()
        
        # Metrics
        
        metrics_buffer = StringIO()
        sys.stdout = metrics_buffer
        evaluator.summarize()
        sys.stdout = STDOUT
        
        metrics = metrics_buffer.getvalue()
        metrics = metrics.split('\n')
        metrics = [m for m in metrics if 'Average' in m]
        metrics_dict = {}
        for metric in metrics:
            name, value = metric.split(' = ')
            metrics_dict[name[1:]] = float(value)
        
        metrics_by_fold.append(metrics_dict)
        
        shutil.rmtree(checkpoints_dir)
          

        # break # Fold
    
    # Aggregate Metrics
    
    index = [f'fold {fold}' for fold in range(1,11)]
    metrics_by_fold = pd.DataFrame(metrics_by_fold, index=index)
    metrics_by_fold.loc['mean'] = metrics_by_fold.mean()
    
    metrics_path = os.path.join(logs_dir, model_name, 'metrics.csv')
    metrics_by_fold.to_csv(metrics_path)
    
    # break # Hyperparameter


(Num Queries, Dim model, Enc-Dec Layers):  (25, 64, 2)
Fold 1


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.decoder.layers.2.final_layer_norm.weight', 'model.backbone.conv_encoder.model.layer1.0.bn2.running_mean', 'model.encoder.layers.4.self_attn.out_proj.weight', 'model.backbone.conv_encoder.model.layer3.1.bn1.bias', 'model.decoder.layers.3.encoder_attn.out_proj.weight', 'model.backbone.conv_encoder.model.layer4.2.conv1.weight', 'model.encoder.layers.5.self_attn.k_proj.weight', 'model.backbone.conv_encoder.model.layer1.2.bn1.running_mean', 'model.backbone.conv_encoder.model.layer4.0.bn1.running_mean', 'model.encoder.layers.4.self_attn.k_proj.weight', 'model.backbone.conv_encoder.model.layer3.1.bn3.running_mean', 'model.decoder.layers.3.fc1.bias', 'model.decoder.layers.4.self_attn.k_proj.weight', 'model.backbone.conv_encoder.model.layer2.0.bn2.weight', 'model.backbone.conv_encoder.model.layer2.3.bn3.running_mean', 'model.backbone.conv_encoder.model.layer3.0.downsam

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type                   | Params
------------------------------------------------------
0 | detr_model | DetrForObjectDetection | 4.7 M 
------------------------------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params
18.980    Total estimated model params size (MB)


<generator object Module.named_parameters at 0x000002535F295E40>


...

c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\utilities\parsing.py:197: UserWarning: Attribute 'detr_model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['detr_model'])`.
  rank_zero_warn(


Accumulating evaluation results...
DONE (t=0.04s).
